In [1]:
import requests
from bs4 import BeautifulSoup
import re
postcode_list = []
header = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

url="https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria"
response = requests.get(url, headers=header)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'^3\d+')
    postcode = soup.find("div",{"class":"col-md-12 column"}).findAll("a",text=pattern)
    for i in postcode :
        postcode_list.append(i.text)

### To save some time it is best to seperate the data in to multipule json file for cleaning. 
#### apartment into postcode_list[0],postcode_list[1:10],postcode_list[11:100],postcode_list[101:200],postcode_list[201:]


In [2]:
# This is for apartment 37minutes
#  This is used to scrape the internal feature of a property from the domain website (Apartment) and seperate in to first 3 folders. 

import requests
from bs4 import BeautifulSoup
# built-in imports
import re
from json import dump
import os
from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
BASE_URL = "https://www.domain.com.au"


header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    

}

page_set=0
for i in (postcode_list[0:1],postcode_list[1:10],postcode_list[10:100]):
    
    page_set+=1
    print(page_set)
    url_links = []
    property_metadata = defaultdict(dict)
    for postcode in i:
        
        count=0
        for page in range(1,52):
            url= f"https://www.domain.com.au/rent/?ptype=apartment&sort=price-desc&page={page}&postcode={postcode}"
            response = requests.get(url,headers = header)
            html_content = response.content
            bs_object = BeautifulSoup(html_content, 'html.parser')
            try :
                index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            except AttributeError:
                break

            for link in index_links:
                
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    count+=1
                    url_links.append(link['href'])
                    
        print(f"{postcode} has {count} property")
        
        
    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:
        response = requests.get(property_url,headers = header)
        html_content = response.content
        bs_object = BeautifulSoup(html_content, 'html.parser')
        
        try: 
            # looks for the header class to get property name
            property_metadata[property_url]['name'] = bs_object \
                .find("h1", {"class": "css-164r41r"}) \
                .text

            # looks for the div containing a summary title for cost
            property_metadata[property_url]['cost_text'] = bs_object \
                .find("div", {"data-testid": "listing-details__summary-title"}) \
                .text

            # extract coordinates from the hyperlink provided
            # i'll let you figure out what this does :P
            property_metadata[property_url]['coordinates'] = [
                float(coord) for coord in re.findall(
                    r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                    bs_object \
                        .find(
                            "a",
                            {"target": "_blank", 'rel': "noopener noreferrer"}
                        ) \
                        .attrs['href']
                )[0].split(',')
            ]

            # get rooms and parking
            rooms = bs_object \
                    .find("div", {"data-testid": "property-features"}) \
                    .findAll("span", {"data-testid": "property-features-text-container"})

            # rooms
            property_metadata[property_url]['rooms'] = [
                re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Bed' in feature.text or 'Bath' in feature.text
            ]
            # parking
            property_metadata[property_url]['parking'] = [
                re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Parking' in feature.text
            ]
            
            a=bs_object.findAll("a",{"class":"css-1f0ty98","title":True})
            b= bs_object.findAll("div", {"data-testid":"fe-co-school-catchment-schoolDistance"})
            distance_list = []
            new_dictionary= defaultdict(dict)
            for i in b:
                distance_list.append(float(i.text.split("km")[0]))
            count= 0
            for i in a :

                name= i["title"]
                new_dictionary[name]= distance_list[count]
                count+=1
            property_metadata[property_url]['school']=new_dictionary
            
            
            
        except AttributeError:
            print(f"Issue with {url}, see raw dump below")
            print(bs_object.text)
            continue
    print(f"start to put in to json{page_set}") 
    if not os.path.exists("../data/landing/web_scraping"):
        os.makedirs("../data/landing/web_scraping")
    with open(f'../data/landing/web_scraping/apartment_{page_set}.json', 'w') as f:
        dump(property_metadata, f)
    
        

1
3000 has 824 property
Issue with https://www.domain.com.au/rent/?ptype=apartment&sort=price-desc&page=43&postcode=3000, see raw dump below
501/263 Franklin Street, Melbourne | Property History & Address Research | Domain Allhomes  Commercial Real Estate NavigationLog inAccountLog inFind a PropertyResearchFind AgentsFor OwnersNewHome LoansNewsCommercialFind a PropertyBuyRentHouse & LandNew HomesRuralShareSearch by schoolResearchProperty Market InsightsProperty Price EstimatesSuburb ProfilesProperty Market AnalysisAuction ResultsSold PropertiesFind Affordable SuburbsGuidesAdviceBuyingInvestingRentingFind AgentsFor OwnersMy PropertiesSellHome LoansNewRefinance Onlinewith Domain Home LoansExplore Home LoansRepayment CalculatorStamp Duty CalculatorRefinance Savings CalculatorEquity CalculatorBorrowing Power CalculatorFixed Rate Expiry CalculatorNewsProperty NewsMoney & MarketsLivingDomain ReviewCommercialHomeProperty Profile501/263 Franklin Street, Melbourne VIC 3000Search for propertySen

In [3]:
# This is for apartment 34minutes
#  This is used to scrape the internal feature of a property from the domain website (Apartment) and seperate in to another 2 folders. 
import requests
from bs4 import BeautifulSoup
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
BASE_URL = "https://www.domain.com.au"


header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    

}

page_set=3
for i in (postcode_list[100:200],postcode_list[200:]):
    
    page_set+=1
    print(page_set)
    url_links = []
    property_metadata = defaultdict(dict)
    for postcode in i:
        
        count=0
        for page in range(1,52):
            url= f"https://www.domain.com.au/rent/?ptype=apartment&sort=price-desc&page={page}&postcode={postcode}"
            response = requests.get(url,headers = header)
            html_content = response.content
            bs_object = BeautifulSoup(html_content, 'html.parser')
            try :
                index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            except AttributeError:
                break

            for link in index_links:
                
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    count+=1
                    url_links.append(link['href'])
                    
        print(f"{postcode} has {count} property")
        
        
    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:
        response = requests.get(property_url,headers = header)
        html_content = response.content
        bs_object = BeautifulSoup(html_content, 'html.parser')
        
        try: 
            # looks for the header class to get property name
            property_metadata[property_url]['name'] = bs_object \
                .find("h1", {"class": "css-164r41r"}) \
                .text

            # looks for the div containing a summary title for cost
            property_metadata[property_url]['cost_text'] = bs_object \
                .find("div", {"data-testid": "listing-details__summary-title"}) \
                .text

            # extract coordinates from the hyperlink provided
            # i'll let you figure out what this does :P
            property_metadata[property_url]['coordinates'] = [
                float(coord) for coord in re.findall(
                    r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                    bs_object \
                        .find(
                            "a",
                            {"target": "_blank", 'rel': "noopener noreferrer"}
                        ) \
                        .attrs['href']
                )[0].split(',')
            ]

            # get rooms and parking
            rooms = bs_object \
                    .find("div", {"data-testid": "property-features"}) \
                    .findAll("span", {"data-testid": "property-features-text-container"})

            # rooms
            property_metadata[property_url]['rooms'] = [
                re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Bed' in feature.text or 'Bath' in feature.text
            ]
            # parking
            property_metadata[property_url]['parking'] = [
                re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Parking' in feature.text
            ]
            
            a=bs_object.findAll("a",{"class":"css-1f0ty98","title":True})
            b= bs_object.findAll("div", {"data-testid":"fe-co-school-catchment-schoolDistance"})
            distance_list = []
            new_dictionary= defaultdict(dict)
            for i in b:
                distance_list.append(float(i.text.split("km")[0]))
            count= 0
            for i in a :

                name= i["title"]
                new_dictionary[name]= distance_list[count]
                count+=1
            property_metadata[property_url]['school']=new_dictionary

        except AttributeError:
            print(f"Issue with {url}, see raw dump below")
            print(bs_object.text)
            continue
    print(f"start to put in to json{page_set}") 
    if not os.path.exists("../data/landing/web_scraping"):
        os.makedirs("../data/landing/web_scraping")
    with open(f'../data/landing/web_scraping/apartment_{page_set}.json', 'w') as f:
        dump(property_metadata, f)
    
        

4
3122 has 75 property
3123 has 34 property
3124 has 23 property
3125 has 25 property
3126 has 7 property
3127 has 12 property
3128 has 71 property
3129 has 6 property
3130 has 12 property
3131 has 11 property
3132 has 8 property
3133 has 4 property
3134 has 10 property
3135 has 7 property
3136 has 9 property
3137 has 1 property
3138 has 2 property
3139 has 1 property
3140 has 1 property
3141 has 164 property
3142 has 39 property
3143 has 17 property
3144 has 12 property
3145 has 51 property
3146 has 33 property
3147 has 6 property
3148 has 4 property
3149 has 11 property
3150 has 18 property
3151 has 22 property
3152 has 5 property
3153 has 11 property
3154 has 0 property
3155 has 12 property
3156 has 5 property
3158 has 0 property
3159 has 0 property
3160 has 0 property
3161 has 39 property
3162 has 22 property
3163 has 60 property
3165 has 8 property
3166 has 25 property
3167 has 6 property
3168 has 53 property
3169 has 15 property
3170 has 5 property
3171 has 7 property
3172 has 1 

### To save some time it is best to seperate the data in to multipule json file for cleaning. 
#### apartment into postcode_list[0:50],postcode_list[51:200],postcode_list[201:400],postcode_list[401:]


In [4]:
# this is for house  43min
#  This is used to scrape the internal feature of a property from the domain website (House) and seperate in to first 2  folders.
import requests
from bs4 import BeautifulSoup
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
BASE_URL = "https://www.domain.com.au"


header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    

}
page_set=0
for i in (postcode_list[0:50],postcode_list[50:200]):
    page_set+=1
    url_links = []
    property_metadata = defaultdict(dict)
    for postcode in i:
        
        count=0
        for page in range(1,52):
            url= f"https://www.domain.com.au/rent/?ptype=house&sort=price-desc&page={page}&postcode={postcode}"
            response = requests.get(url,headers = header)
            html_content = response.content
            bs_object = BeautifulSoup(html_content, 'html.parser')

            
            
            
            try :
                index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            except AttributeError:
                break

            for link in index_links:
                
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    count+=1
                    url_links.append(link['href'])
                    
        print(f"{postcode} has {count} property")
    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:
        response = requests.get(property_url,headers = header)
        html_content = response.content
        bs_object = BeautifulSoup(html_content, 'html.parser')
        
        try: 
            # looks for the header class to get property name
            property_metadata[property_url]['name'] = bs_object \
                .find("h1", {"class": "css-164r41r"}) \
                .text

            # looks for the div containing a summary title for cost
            property_metadata[property_url]['cost_text'] = bs_object \
                .find("div", {"data-testid": "listing-details__summary-title"}) \
                .text

            # extract coordinates from the hyperlink provided
            # i'll let you figure out what this does :P
            property_metadata[property_url]['coordinates'] = [
                float(coord) for coord in re.findall(
                    r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                    bs_object \
                        .find(
                            "a",
                            {"target": "_blank", 'rel': "noopener noreferrer"}
                        ) \
                        .attrs['href']
                )[0].split(',')
            ]

            # get rooms and parking
            rooms = bs_object \
                    .find("div", {"data-testid": "property-features"}) \
                    .findAll("span", {"data-testid": "property-features-text-container"})

            # rooms
            property_metadata[property_url]['rooms'] = [
                re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Bed' in feature.text or 'Bath' in feature.text
            ]
            # parking
            property_metadata[property_url]['parking'] = [
                re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Parking' in feature.text
            ]
            
            a=bs_object.findAll("a",{"class":"css-1f0ty98","title":True})
            b= bs_object.findAll("div", {"data-testid":"fe-co-school-catchment-schoolDistance"})
            distance_list = []
            new_dictionary= defaultdict(dict)
            for i in b:
                distance_list.append(float(i.text.split("km")[0]))
            count= 0
            for i in a :

                name= i["title"]
                new_dictionary[name]= distance_list[count]
                count+=1
            property_metadata[property_url]['school']=new_dictionary
            

        except AttributeError:
            print(f"Issue with {url}, see raw dump below")
            print(bs_object.text)
            continue
    # output to example json in data/raw/
    if not os.path.exists("../data/landing/web_scraping"):
        os.makedirs("../data/landing/web_scraping")
    with open(f'../data/landing/web_scraping/house_{page_set}.json', 'w') as f:
        dump(property_metadata, f)


3000 has 42 property
3002 has 3 property
3003 has 6 property
3004 has 1 property
3006 has 2 property
3008 has 9 property
3011 has 32 property
3012 has 27 property
3013 has 20 property
3015 has 20 property
3016 has 9 property
3018 has 2 property
3019 has 4 property
3020 has 24 property
3021 has 31 property
3022 has 5 property
3023 has 40 property
3024 has 105 property
3025 has 7 property
3026 has 5 property
3027 has 10 property
3028 has 17 property
3029 has 183 property
3030 has 141 property
3031 has 27 property
3032 has 25 property
3033 has 10 property
3034 has 14 property
3036 has 3 property
3037 has 23 property
3038 has 6 property
3039 has 6 property
3040 has 22 property
3041 has 8 property
3042 has 15 property
3043 has 21 property
3044 has 30 property
3045 has 0 property
3046 has 29 property
3047 has 16 property
3048 has 8 property
3049 has 6 property
3051 has 26 property
3052 has 5 property
3053 has 12 property
3054 has 24 property
3055 has 14 property
3056 has 34 property
3057 has

In [5]:
# this is for house  34min
#  This is used to scrape the internal feature of a property from the domain website (House) and seperate in to another 2 folders.
import requests
from bs4 import BeautifulSoup
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
BASE_URL = "https://www.domain.com.au"



header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    

}
page_set=3
for i in (postcode_list[200:401],postcode_list[400:]):
    page_set+=1
    url_links = []
    property_metadata = defaultdict(dict)
    for postcode in i:
        
        count=0
        for page in range(1,52):
            url= f"https://www.domain.com.au/rent/?ptype=house&sort=price-desc&page={page}&postcode={postcode}"
            response = requests.get(url,headers = header)
            html_content = response.content
            bs_object = BeautifulSoup(html_content, 'html.parser')

            
            
            
            try :
                index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            except AttributeError:
                break

            for link in index_links:
                
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    count+=1
                    url_links.append(link['href'])
                    
        print(f"{postcode} has {count} property")
    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:
        response = requests.get(property_url,headers = header)
        html_content = response.content
        bs_object = BeautifulSoup(html_content, 'html.parser')
        
        try: 
            # looks for the header class to get property name
            property_metadata[property_url]['name'] = bs_object \
                .find("h1", {"class": "css-164r41r"}) \
                .text

            # looks for the div containing a summary title for cost
            property_metadata[property_url]['cost_text'] = bs_object \
                .find("div", {"data-testid": "listing-details__summary-title"}) \
                .text

            # extract coordinates from the hyperlink provided
            # i'll let you figure out what this does :P
            property_metadata[property_url]['coordinates'] = [
                float(coord) for coord in re.findall(
                    r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                    bs_object \
                        .find(
                            "a",
                            {"target": "_blank", 'rel': "noopener noreferrer"}
                        ) \
                        .attrs['href']
                )[0].split(',')
            ]

            # get rooms and parking
            rooms = bs_object \
                    .find("div", {"data-testid": "property-features"}) \
                    .findAll("span", {"data-testid": "property-features-text-container"})

            # rooms
            property_metadata[property_url]['rooms'] = [
                re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Bed' in feature.text or 'Bath' in feature.text
            ]
            # parking
            property_metadata[property_url]['parking'] = [
                re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms
                if 'Parking' in feature.text
            ]
            
            a=bs_object.findAll("a",{"class":"css-1f0ty98","title":True})
            b= bs_object.findAll("div", {"data-testid":"fe-co-school-catchment-schoolDistance"})
            distance_list = []
            new_dictionary= defaultdict(dict)
            for i in b:
                distance_list.append(float(i.text.split("km")[0]))
            count= 0
            for i in a :

                name= i["title"]
                new_dictionary[name]= distance_list[count]
                count+=1
            property_metadata[property_url]['school']=new_dictionary
            
            
        except AttributeError:
            print(f"Issue with {url}, see raw dump below")
            print(bs_object.text)
            continue
    # output to example json in data/raw/
    if not os.path.exists("../data/landing/web_scraping"):
        os.makedirs("../data/landing/web_scraping")
    with open(f'../data/landing/web_scraping/house_{page_set}.json', 'w') as f:
        dump(property_metadata, f)


3230 has 3 property
3231 has 1 property
3232 has 0 property
3233 has 6 property
3234 has 0 property
3235 has 1 property
3236 has 1 property
3237 has 0 property
3238 has 0 property
3239 has 0 property
3240 has 2 property
3241 has 3 property
3242 has 1 property
3243 has 0 property
3249 has 2 property
3250 has 5 property
3251 has 0 property
3254 has 1 property
3260 has 3 property
3264 has 1 property
3265 has 0 property
3266 has 2 property
3267 has 0 property
3268 has 0 property
3269 has 0 property
3270 has 1 property
3271 has 0 property
3272 has 3 property
3273 has 0 property
3274 has 0 property
3275 has 0 property
3276 has 0 property
3277 has 0 property
3278 has 0 property
3279 has 0 property
3280 has 27 property
3281 has 0 property
3282 has 0 property
3283 has 0 property
3284 has 4 property
3285 has 0 property
3286 has 0 property
3287 has 0 property
3289 has 1 property
3292 has 0 property
3293 has 0 property
3294 has 1 property
3300 has 5 property
3301 has 0 property
3302 has 0 property

#### town house 

In [7]:
# this is for town-house  25min
#  This is used to scrape the internal feature of a property from the domain website (Town house) in to 1 folders.
import requests
from bs4 import BeautifulSoup
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
BASE_URL = "https://www.domain.com.au"
url_links = []
property_metadata = defaultdict(dict)

header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    

}
for postcode in postcode_list:
    
    count=0
    for page in range(1,52):
        url= f"https://www.domain.com.au/rent/?ptype=town-house&sort=price-desc&page={page}&postcode={postcode}"
        response = requests.get(url,headers = header)
        html_content = response.content
        bs_object = BeautifulSoup(html_content, 'html.parser')

        
        
        
        try :
            index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                 "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )
        except AttributeError:
            break

        for link in index_links:
            
            # if its a property address, add it to the list
            if 'address' in link['class']:
                count+=1
                url_links.append(link['href'])
                
    print(f"{postcode} has {count} property")
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    response = requests.get(property_url,headers = header)
    html_content = response.content
    bs_object = BeautifulSoup(html_content, 'html.parser')
    
    try: 
        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text

        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferrer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]

        # get rooms and parking
        rooms = bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})

        # rooms
        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms
            if 'Bed' in feature.text or 'Bath' in feature.text
        ]
        # parking
        property_metadata[property_url]['parking'] = [
            re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms
            if 'Parking' in feature.text
        ]
        a=bs_object.findAll("a",{"class":"css-1f0ty98","title":True})
        b= bs_object.findAll("div", {"data-testid":"fe-co-school-catchment-schoolDistance"})
        distance_list = []
        new_dictionary= defaultdict(dict)
        for i in b:
            distance_list.append(float(i.text.split("km")[0]))
        count= 0
        for i in a :
            name= i["title"]
            new_dictionary[name]= distance_list[count]
            count+=1
        property_metadata[property_url]['school']=new_dictionary
        
    except AttributeError:
        print(f"Issue with {url}, see raw dump below")
        print(bs_object.text)
        continue

# output to example json in data/raw/
if not os.path.exists("../data/landing/web_scraping"):
        os.makedirs("../data/landing/web_scraping")
with open('../data/landing/web_scraping/town-house.json', 'w') as f:
    dump(property_metadata, f)

3000 has 0 property
3002 has 1 property
3003 has 2 property
3004 has 0 property
3006 has 0 property
3008 has 2 property
3011 has 9 property
3012 has 19 property
3013 has 8 property
3015 has 6 property
3016 has 2 property
3018 has 1 property
3019 has 5 property
3020 has 6 property
3021 has 10 property
3022 has 1 property
3023 has 3 property
3024 has 3 property
3025 has 4 property
3026 has 1 property
3027 has 1 property
3028 has 2 property
3029 has 9 property
3030 has 9 property
3031 has 4 property
3032 has 15 property
3033 has 6 property
3034 has 8 property
3036 has 0 property
3037 has 11 property
3038 has 3 property
3039 has 4 property
3040 has 13 property
3041 has 5 property
3042 has 14 property
3043 has 3 property
3044 has 23 property
3045 has 0 property
3046 has 38 property
3047 has 6 property
3048 has 0 property
3049 has 3 property
3051 has 3 property
3052 has 3 property
3053 has 3 property
3054 has 3 property
3055 has 7 property
3056 has 13 property
3057 has 7 property
3058 has 15